
![JupyterGames](./JupyterGames.png)


## Motivation

Making simple video games has always been a great way to learn programming for beginner.
While jupyter is the defacto standard for interactive programming in data science and machine learning,
they are not widely used for teaching programming and game development.


## The canvas

A drawable surface is the most basic requirement for any game development. In the web world, this is usually provided by the HTML5 `<canvas>` element.
In Jupyter, there are several ways to create and manipulate a canvas. The most popular one is the `ipycanvas` package, which provides a high-level API for creating and manipulating canvases in Jupyter notebooks.
However, `ipycanvas` is tied to the jupyter protocol, which makes it difficult to create a smooth gaming experience.

The jupyter protocol is a fire and forget protocol. Once a cell is executed, the kernel sends the output to the frontend and forgets about it.
This works well for data science and machine learning, where the output is usually a static image or text.
However, for game development, we need a way to continuously update the output at a fixed frame rate (e.g., 60 frames per second). 
While `ipycanvas` provides a way to update the canvas, it is impossible with ipycanvas to know when the frontend has actually rendered the updated canvas. This can lead to situations where the kernel is sending updates faster than the frontend can render them, leading to a choppy and unresponsive gaming experience.


## JupiterLite

[JupyterLite](https://jupyterlite.readthedocs.io/en/latest/) is a Jupyter distribution that runs entirely in the browser. It uses kernels compiled to WebAssembly like xeus-python, pyodide, xeus-r and even xeus-cpp, to run code directly in the browser without any server backend.
Jupyterlite supports widgets like `ipywidgets` and `ipycanvas` but is also tied to the jupyter protocol.

##

## The OffscreenCanvas 

The HTML5 OffscreenCanvas API provides a way to create a canvas that can be rendered offscreen, i.e., without being attached to the DOM. This allows us to create a canvas in the frontend / main-ui thread, and then transfer it to a web worker for rendering. 
Using the OffscreenCanvas API, we can create a custom canvas widget that creates an OffscreenCanvas in the frontend, transfers it to the kernel running in a web worker, and then uses it for rendering.
Since kernels like xeus-python provide a Javascript foreign function interface, we can directly call Javascript functions from Python , and use them to directely call the OffscreenCanvas API. Hence we can create a smooth gaming experience and render to the canvas at a fixed frame rate. That way, it is impossible to send updates faster than the frontend can render them, since the calls to the OffscreenCanvas API are blocking until the rendering is done.
That way we can bypass the limitations of the jupyter protocol and create a smooth gaming experience in JupyterLite.

We integrated the OffscreenCanvas API directely into the `ipycanvas` package and created a custom canvas widget called `OffscreenCanvas` which provides the same API as the regular `Canvas` widget from `ipycanvas`, but uses the OffscreenCanvas API for rendering.



## Overcome the protocol limitations for regular Jupyter

Since the OffscreenCanvas API is not available in regular Jupyter notebooks, we need to find a way to overcome the limitations of the jupyter protocol.
While its impossible with ipycanvas existing API to know when the canvas has been rendered in the frontned for the reason mentioned above, we can work around this limitation by creating a custom frontend extension that uses the following trick.

Asuming we want to know within a the exection of a single cell when some value of a widget has changed in the frontend,
we can utilize the following snippet that creates an asyncio.Future that will be set once the value of the widget changes in the frontend.

```python
import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future
```

As an example, the following code creates an IntSlider widget and an async function that waits for the value of the slider to change.

```python
from ipywidgets import IntSlider
slider = IntSlider()

async def f():
    for i in range(5):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider
```

In [ ]:
import asyncio
from ipywidgets import IntSlider

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

slider = IntSlider()

async def f():
    for i in range(5):
        print('did work %s'%i)
        x = await wait_for_change(slider, 'value')
        print('async function continued with value %s'%x)
asyncio.ensure_future(f())

slider

Using the above trick, we can create a custom canvas widget that will notify the kernel once the canvas has been rendered in the frontend. This way, we can create a game loop that runs at a fixed frame rate and only updates the canvas once the previous frame has been rendered.

# Box2d

[Box2D](https://box2d.org/) is a popular 2D physics engine that is widely used in game development. It provides a simple and efficient way to simulate physics in 2D games.
Recently, there has been a major update to the Box2D codebase, namely Box2D 3.0 which introduces several new features and dramatic performance improvements over the previous versions.

## Python wrapper for Box2D 3.0
We have created a Python wrapper for Box2D 3.0 called [pyb2d3](https://github.com/DerThorsten/pyb2d3/) that allows you to use Box2D 3.0 in Python. The wrapper is built using [nanobind](https://nanobind.github.io/) and provides a simple and efficient way to use Box2D 3 in Python.
To be able to use pyb2d3 in jupyterlite  we created a wasm build of pyb2d3 using [emscripten](https://emscripten.org/) and the  [emscripten-forge](https://emscripten-forge.org/) conda [distribution](https://prefix.dev/channels/emscripten-forge-dev).

## Box2D sandbox

Box2d and its python wrapper pyb2d3 do not provide any rendering capabilities out of the box. However both Box2D and pyb2d3 provide a "sandbox" playground that allows you to quickly create simple Box2D which are automatically rendered.

For pyb2d3, we have created a custom package for this sandbox called `pyb2d3-sandbox` with various frontends, including a jupyter notebook frontend that uses the custom canvas widget described above to provide a smooth gaming experience in jupyter notebooks.

These frontends are:

 * pyb2d3-sandbox-opengl: A desktop OpenGL frontend for pyb2d3-sandbox using PyOpenGL and dearimgui
 * pyb2d3-sandbox-pygame: A desktop pygame frontend for pyb2d3-sandbox using pygame for rendering
 * pyb2d3-sandbox-ipycanvas: A Jupyter notebook frontend for pyb2d3-sandbox using the custom OffscreenCanvas widget when in JupyterLite
 * pyb2d3-sandbox-jupyter: A Jupyter notebook frontend for pyb2d3-sandbox using `wait_for_change` trick when in regular Jupyter notebooks


# Examples

All the examples can also be found, edited and run in [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/)

## Netwons Cradle

Lets start with the classic Newtons Cradle example. The example can be restarted by pressing the stop-like button in the lower left corner of the canvas.
Mouse or touch interaction can be used to drag the balls arround.

```python
world = b2d.World(gravity=(0, -10))
n_balls = 10
radius = 1
actual_radius = radius * 0.85 # make balls a  bit smaller st. 
                              # there is a little space between them
rope_length = 10
diameter = radius * 2

for i in range(n_balls):
    x = diameter * i
    y_ball = 0
    y_rope = rope_length

    # create dynamic body for the ball
    ball_body = world.create_dynamic_body(
        position=(x, y_ball), linear_damping=0.1, angular_damping=0.0
    )
    ball_body.awake = True
    # create circle shape for the ball
    material = b2d.surface_material(
        restitution=1.0,
        friction=0.0,
        custom_color=b2d.hex_color(100, 0, 200),
    )
    ball_body.create_shape(
        b2d.shape_def(density=1, material=material),
        b2d.circle(radius=actual_radius),
    )

    # create a rope anchor for the balls
    anchor_pos = (x, y_rope)
    anchor_body_id = world.create_static_body(position=anchor_pos)

    self.world.create_distance_joint(
        body_a=ball_body,
        body_b=anchor_body_id,
        length=self.rope_length,
        enable_spring=False,
    )

impulse = (-10, 0)
ball_body.apply_linear_impulse_to_center(impulse, wake=True)
```

In [ ]:
import pyb2d3 as b2d
from pyb2d3_sandbox import SampleBase


class NewtonsCradle(SampleBase):
    def __init__(self, frontend, settings):
        super().__init__(frontend, settings)

        # physical world
        self.n_balls = 10
        self.radius = 1
        self.actual_radius = self.radius * 0.85
        self.rope_length = 10

        diameter = self.radius * 2
        for i in range(self.n_balls):
            x = diameter * i
            y_ball = 0
            y_rope = self.rope_length

            # create dynamic body for the ball
            ball_body = self.world.create_dynamic_body(
                position=(x, y_ball), linear_damping=0.1, angular_damping=0.0
            )
            ball_body.awake = True
            # create circle shape for the ball
            material = b2d.surface_material(
                restitution=1.0,
                friction=0.0,
                custom_color=b2d.hex_color(100, 0, 200),
            )
            ball_body.create_shape(
                b2d.shape_def(density=1, material=material),
                b2d.circle(radius=self.actual_radius),
            )

            # create a rope anchor for the balls
            anchor_pos = (x, y_rope)
            anchor_body_id = self.world.create_static_body(position=anchor_pos)

            self.world.create_distance_joint(
                body_a=ball_body,
                body_b=anchor_body_id,
                length=self.rope_length,
                enable_spring=False,
            )

        impulse = (-10, 0)
        ball_body.apply_linear_impulse_to_center(impulse, wake=True)

    def aabb(self):
        return b2d.aabb(
            lower_bound=(-(self.rope_length + 2 * self.radius), 0),
            upper_bound=(
                self.n_balls * self.radius * 2 + self.rope_length,
                self.rope_length + 2 * self.radius,
            ),
        )


if __name__  == "__main__":
    NewtonsCradle.run(frontend_settings=dict(simple_ui=True, autostart=True))

## Billiard

The source code of the billiard example can be found in `billiard.ipynb` at the [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/) deployment.
To start the example, press the play button. Note that this will pause the simulation of all other running examples.
To shoot a ball, click into a ball and drag the mouse / finger backwards. On release, the ball will shot.

In [ ]:
import pyb2d3 as b2d
from pyb2d3_sandbox import SampleBase

import numpy as np
import math

# from dataclass  import dataclass
from dataclasses import dataclass
import random

import enum


@dataclass
class Ball:
    color: tuple[int, int, int]
    is_half: bool = False
    is_white: bool = False
    is_black: bool = False
    body: b2d.Body = None


# enum for state of game
class GameState(enum.Enum):
    WAITING_FOR_BALL_SELECTION = enum.auto()
    WAITING_FOR_SHOT = enum.auto()
    WAITING_FOR_BALLS_TO_REST = enum.auto()


class Billiard(SampleBase):
    def __init__(self, frontend, settings):
        super().__init__(frontend, settings.set_gravity((0, 0)))

        # billiard table
        w = 8
        h = w / 2
        self.width_lower_bound = w
        self.height_lower_bound = h

        # top_left_corner
        top_left_corner = (-w / 2, h / 2)

        # pocket radius
        r = 0.5
        self.pocket_radius = r

        md = r * math.sqrt(2) * 1.3
        cd = r * 1.3

        self.pocket_centers = []

        # start with left pocket arc
        start = (top_left_corner[0], top_left_corner[1])
        builder = b2d.PathBuilder(start)
        pocket_center = builder.arc_to(
            delta=(-cd, -cd), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move down by height
        builder.line_to(down=h)

        # add bottom left corner pocket
        pocket_center = builder.arc_to(
            delta=(cd, -cd), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move right by width /2
        builder.line_to(right=w / 2)
        # add bottom middle pocket
        pocket_center = builder.arc_to(
            delta=(md, 0), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move right by width /2
        builder.line_to(right=w / 2)
        # add bottom right corner pocket
        pocket_center = builder.arc_to(
            delta=(cd, cd), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move up by height
        builder.line_to(up=h)
        # add top right corner pocket
        pocket_center = builder.arc_to(
            delta=(-cd, cd), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)

        # move left by width/2
        builder.line_to(left=w / 2)
        # add top middle pocket
        pocket_center = builder.arc_to(
            delta=(-md, 0), radius=r, clockwise=False, segments=20, major_arc=True
        )
        self.pocket_centers.append(pocket_center)
        # move left by width/2
        builder.line_to(left=w / 2)

        assert len(self.pocket_centers) == 6, "There should be 6 pocket centers"

        table_center = (np.array(self.pocket_centers[2]) + np.array(self.pocket_centers[5])) / 2
        table_center = (float(table_center[0]), float(table_center[1]))
        self.table_center = table_center

        # add 15 balls
        self.balls = []
        self.ball_radius = 0.2

        def create_ball(position, color):
            material = b2d.surface_material(
                custom_color=b2d.hex_color(color[0], color[1], color[2]),
                restitution=1.0,
            )
            ball = self.world.create_dynamic_body(
                position=position, linear_damping=0.8, fixed_rotation=True
            )
            ball.create_shape(b2d.shape_def(material=material), b2d.circle(radius=self.ball_radius))
            return ball

        billiard_base_colors = [
            (255, 255, 0),  # 1 - Yellow
            (0, 0, 255),  # 2 - Blue
            (255, 0, 0),  # 3 - Red
            (128, 0, 128),  # 4 - Purple
            (255, 165, 0),  # 5 - Orange
            (0, 128, 0),  # 6 - Green
            (128, 0, 0),  # 7 - Maroon (Burgundy)
        ]
        # create a list of all balls (except white and black)
        for i, color in enumerate(billiard_base_colors):
            self.balls.append(Ball(color=color, is_half=False))
            self.balls.append(Ball(color=color, is_half=True))

        # shuffle the balls
        random.shuffle(self.balls[1:])  # keep the first ball (yellow) in place

        self.balls.insert(4, Ball(color=(0, 0, 1), is_black=True))

        # add a white ball as last ball
        self.balls.append(Ball(color=(255, 255, 255), is_white=True))

        ball_index = 0
        for x in range(5):
            n_balls = x + 1
            # pos_x = x * self.ball_radius *
            pos_x = x * self.ball_radius * math.sqrt(3)
            offset_y = -x * self.ball_radius

            for y in range(n_balls):
                ball_item = self.balls[ball_index]
                pos_y = offset_y + y * self.ball_radius * 2
                ball_body = create_ball(
                    position=(table_center[0] + pos_x, table_center[1] + pos_y),
                    color=ball_item.color,
                )
                self.balls[ball_index].body = ball_body
                ball_index += 1

        # white ball
        white_ball = create_ball(
            position=(table_center[0] - w / 4, table_center[1]),
            color=self.balls[-1].color,
        )
        self.balls[-1].body = white_ball

        self.outline = np.array(builder.points)

        material = b2d.surface_material(
            restitution=1.0,
            custom_color=b2d.hex_color(10, 150, 10),
        )
        self.anchor_body.create_chain(
            builder.chain_def(is_loop=True, reverse=True, material=material)
        )

        # state of the game
        self.game_state = GameState.WAITING_FOR_BALL_SELECTION
        self.marked_point_on_white_ball = None
        self.aim_point = None

        # in case of a headless frontend we do one shot
        if self.frontend.settings.headless:
            # we create a mouse joint for the white ball
            self.balls[-1].body.apply_linear_impulse_to_center((3.5, 0.1), wake=True)
            self.game_state = GameState.WAITING_FOR_BALLS_TO_REST

    def on_mouse_down(self, event):
        if self.game_state == GameState.WAITING_FOR_BALL_SELECTION:
            # check if mouse is over **the white ball**
            ball_shape = self.balls[-1].body.shapes[0]
            world_pos = event.world_position
            if ball_shape.test_point(point=world_pos):
                self.marked_point_on_white_ball = world_pos
                self.aim_point = world_pos
                self.game_state = GameState.WAITING_FOR_SHOT

    def on_mouse_move(self, event):
        if self.game_state == GameState.WAITING_FOR_SHOT:
            self.aim_point = event.world_position

    def on_mouse_up(self, event):
        if self.game_state == GameState.WAITING_FOR_SHOT:
            # shoot the white ball
            # get the impulse vector from the marked point on the white ball to the force selection point
            ball_pos = self.balls[-1].body.position
            impulse_vec = -(np.array(self.aim_point) - np.array(ball_pos))
            impulse_vec = (
                float(impulse_vec[0]),
                float(impulse_vec[1]),
            )  # convert to tuple
            # apply the force as impulse
            self.balls[-1].body.apply_linear_impulse(impulse_vec, self.marked_point_on_white_ball)
            self.game_state = GameState.WAITING_FOR_BALLS_TO_REST
            self.marked_point_on_white_ball = None
            self.aim_point = None

    def pre_update(self, dt):
        if self.game_state == GameState.WAITING_FOR_BALLS_TO_REST:
            all_rest = True
            for ball in self.balls:
                body = ball.body
                mag = body.linear_velocity_magnitude()
                if mag > 0.0001:  # threshold for resting
                    all_rest = False
                    break
            if all_rest:
                self.game_state = GameState.WAITING_FOR_BALL_SELECTION
                self.marked_point_on_white_ball = None
                self.aim_point = None

        # check if any ball is in a pocket
        to_be_removed = []
        dr = self.pocket_radius - self.ball_radius
        for ball in self.balls:
            body = ball.body
            for pocket_center in self.pocket_centers:
                if np.linalg.norm(np.array(body.position) - np.array(pocket_center)) < dr:
                    to_be_removed.append(ball)
                    break

        for ball in to_be_removed:
            if ball.is_white or ball.is_black:
                self.frontend.set_sample(type(self), self.settings)
            if self.mouse_joint_body is not None:
                if ball.body == self.mouse_joint_body:
                    self.destroy_mouse_joint()
            self.balls.remove(ball)
            ball.body.destroy()

    def pre_debug_draw(self):
        for pocket_center in self.pocket_centers:
            t = b2d.transform(pocket_center)
            self.debug_draw.draw_solid_circle(
                transform=t,
                radius=0.5,
                color=(0, 0, 0),
            )

        # self.debug_draw.draw_polygon(points=self.outline, color=(10, 150, 10))

    def post_debug_draw(self):
        for ball in self.balls:
            self.debug_draw.draw_solid_circle(
                transform=ball.body.transform,
                radius=self.ball_radius,
                color=ball.color,
            )
            if ball.is_half:
                self.debug_draw.draw_solid_circle(
                    transform=ball.body.transform,
                    radius=self.ball_radius / 2,
                    color=(255, 255, 255),
                )
        if self.marked_point_on_white_ball is not None:
            self.debug_draw.draw_solid_circle(
                transform=b2d.transform(self.marked_point_on_white_ball),
                radius=0.1,
                color=(255, 0, 0),
            )

            if self.aim_point is not None:
                self.debug_draw.draw_segment(
                    self.marked_point_on_white_ball, self.aim_point, color=(255, 0, 0)
                )

    def aabb(self):
        hx = self.width_lower_bound / 2
        hy = self.height_lower_bound / 2
        margin = hx * 1.5
        center = self.table_center
        return b2d.aabb(
            lower_bound=(center[0] - (hx + margin), center[1] - (hy + margin)),
            upper_bound=(center[0] + (hx + margin), center[1] + (hy + margin)),
        )


if __name__  == "__main__":
    Billiard.run(frontend_settings=dict(simple_ui=True, autostart=False))

## Ragdoll

While this is not a game, its still fun to man-handle the ragdoll using mouse or touch interaction.
A double click will trigger an explosion that will fling the ragdoll parts arround. A tripple click will trigger an implosion that will suck the ragdoll parts towards the click position.
The source code of this example can be found in `ragdoll.ipynb` at the [notebook.link](https://notebook.link/@DerThorsten/jupyter-games/)

In [ ]:
import pyb2d3 as b2d
from pyb2d3_sandbox import SampleBase
import random

from examples_common import Ragdoll as RagdollComposit


class Ragdoll(SampleBase):
    def __init__(self, frontend, settings):
        super().__init__(frontend, settings)

        self.outer_box_radius = 30
        self.box_body = self.world.create_static_body(position=(0, 0))
        self.box_body.create_chain(
            b2d.chain_box(center=(0, 0), hx=self.outer_box_radius, hy=self.outer_box_radius)
        )

        def rand_pos():
            margin = 2
            r = self.outer_box_radius - margin
            return (random.uniform(-r, r), random.uniform(-r, 0))

        num_bodies = 15
        for _ in range(num_bodies):
            # ragdoll at the center
            self.ragdoll = RagdollComposit(
                scale=7.0,
                world=self.world,
                position=rand_pos(),
                colorize=True,
                hertz=0.1,
            )

        # only relevant for a headless ui
        self._exploded = False

    def pre_update(self, dt):
        if self.frontend.settings.headless and self.world_time > 2 and not self._exploded:
            self._exploded = True
            self.world.explode(
                position=(0, -self.outer_box_radius), radius=20, impulse_per_length=30
            )

    def on_double_click(self, event):
        self.world.explode(position=event.world_position, radius=20, impulse_per_length=30)

    def on_triple_click(self, event):
        self.world.explode(position=event.world_position, radius=20, impulse_per_length=-30)

    def aabb(self):
        return b2d.aabb(
            lower_bound=(-self.outer_box_radius, -self.outer_box_radius),
            upper_bound=(self.outer_box_radius, self.outer_box_radius),
        )


if __name__ == "__main__":
    Ragdoll.run(frontend_settings=dict(simple_ui=True, autostart=False))